## Use the model to predict what type of song to generate

#### Things we want to request from the user [model input]:
1) Current mood (rate from 1 (very bad) to 5 (very good)):’mood'
2) mood after listening to song. Choose from the following: 'amazement', 'solemnity', 'tenderness', 'nostalgia', 'calmness', 'power', 'joyful_activation', 'tension', 'sadness'
3) Age of user:’ age'

##### Things to extract from user listening history:
4) 'avg_bpm_listened'
5) 'most_listened_genre'
6) 'listening_frequency' choose from (hourly, daily, weekly, monthly)

#### Generate a song based on [model output]: 
1) 'Genre' 
2) 'Key' 
3) 'BPM'

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import random
import numpy as np

In [3]:
class NeuralNetwork(nn.Module):
    def __init__(self, input_size, output_size):
        super(NeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(input_size, 128)  
        self.fc2 = nn.Linear(128, 64)          
        self.fc3 = nn.Linear(64, 32)           
        self.fc4 = nn.Linear(32, output_size)  

    def forward(self, x):
        x = F.relu(self.fc1(x))  
        x = F.relu(self.fc2(x))  
        x = F.relu(self.fc3(x))  
        x = self.fc4(x)          
        return x

In [4]:
model = torch.load('my_model.pth')
model.eval()

NeuralNetwork(
  (fc1): Linear(in_features=22, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=32, bias=True)
  (fc4): Linear(in_features=32, out_features=29, bias=True)
)

In [22]:
inputs = pd.read_csv('inputs.csv')
outputs = pd.read_csv('targets.csv')

In [90]:
random_row = inputs.sample(n=1)
random_output = outputs.sample(n=1)
input_tensor = torch.tensor(random_row.values, dtype=torch.float32)

In [91]:
random_row.columns

Index(['amazement', 'solemnity', 'tenderness', 'nostalgia', 'calmness',
       'power', 'joyful_activation', 'tension', 'sadness', 'mood', 'age',
       'avg_bpm_listened', 'most_listened_genre_Classical',
       'most_listened_genre_Electronic', 'most_listened_genre_Hip-hop',
       'most_listened_genre_Jazz', 'most_listened_genre_Pop',
       'most_listened_genre_Rock', 'listening_frequency_daily',
       'listening_frequency_hourly', 'listening_frequency_monthly',
       'listening_frequency_weekly'],
      dtype='object')

In [92]:
# Pass the fake input tensor through the model to get predictions
with torch.no_grad():
    predictions = model(input_tensor)

# Convert predictions to a readable format 
predicted_values = predictions.numpy()

In [83]:
def decode_predictions(predictions, target_vector_encoded):
    """
    Decodes the output of the neural network model from one-hot encoded arrays to a readable dictionary format.

    :param predictions: Numpy array of model predictions.
    :param input_vector_encoded: DataFrame of the encoded input features.
    :param target_vector_encoded: DataFrame of the encoded target features.
    :return: A dictionary containing readable model predictions.
    """
    # Initialize a dictionary to hold the decoded results
    decoded_results = {
        'Genre': '',
        'Key': '',
        'BPM': 0
    }

    # Get the list of encoded columns for Genre and Key
    genre_cols = [col for col in target_vector_encoded if col.startswith('Genre_')]
    key_cols = [col for col in target_vector_encoded if col.startswith('Key_')]

    # Extract the portions of the prediction associated with Genre and Key
    genre_predictions = predictions[0, [target_vector_encoded.columns.get_loc(col) for col in genre_cols]]
    key_predictions = predictions[0, [target_vector_encoded.columns.get_loc(col) for col in key_cols]]

    # Identify the index of the highest probability in each category, which represents the model's prediction
    predicted_genre = genre_cols[np.argmax(genre_predictions)].split('_', 1)[1]  # Extracting the actual genre
    predicted_key = key_cols[np.argmax(key_predictions)].split('_', 1)[1]  # Extracting the actual key

    # Since BPM is a numerical value, directly use the corresponding prediction
    # Assuming the BPM is at a fixed index in your output layer - adjust if different
    bpm_index = target_vector_encoded.columns.get_loc('BPM')
    predicted_bpm = predictions[0, bpm_index]

    # Update the results dictionary
    decoded_results['Genre'] = predicted_genre
    decoded_results['Key'] = predicted_key
    decoded_results['BPM'] = int(round(predicted_bpm)) 

    return decoded_results

In [89]:
decode_predictions(predicted_values, random_output)

{'Genre': 'classical', 'Key': 'G Major', 'BPM': 121}